## Импорт и загрузка данных

In [379]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import time

# multiprocessing
from multiprocessing.pool import Pool
from multiprocessing.pool import ThreadPool

In [383]:
domain = pd.read_csv('../datasets/domain.csv')

## Сбор данных

In [381]:
def find_lang_new(url):
    '''
    Функция 
    - принимает адрес домена
    - подставляет к нему http:// или https://
    - возвращает информацию о языке сайта указанном в атрибуте lang тега html, если такой есть
    - если возникает какая-то ошибка, то возвращает имя данной ошибки 
    '''
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}
    
    try:
        html = requests.get('http://'+ str(url), timeout=30, headers=headers).text
        soup = BeautifulSoup(html, 'html.parser')
        lang = soup.html['lang']
        return lang

    # обработка ошибки с отсутствием доступа по http
    except requests.exceptions.InvalidURL:
        html = requests.get('https://'+ str(url), timeout=5, headers=headers).text
        soup = BeautifulSoup(html, 'html.parser')
        lang = soup.html['lang']
        return lang

    # все остальные ошибки
    except Exception as exception:
        return type(exception).__name__

In [384]:
# Запуск поиска языка в многопоточном режиме
with ThreadPool(processes=100) as p:
    domain['lang'] = tqdm(p.imap(find_lang_new, domain['domain']), total=len(domain))

## Предобработка данных

Так как парсер был не самым идеальным и собрать 100% данных не получилось, то придется поработать руками прежде чем переходить к непосредственному анализу и выводам. Так как перед сохранением в файл не было никаких подготовительных процедур, то проведем их сейчас.

In [386]:
domain['lang'] = domain['lang'].str.lower()

### Обработка ошибок
Глобально все ошибки возникшие при сборе данных можно поделить на две части: 
- пустые значения - с сайтом удалось связаться, найти тег `html`, найти атрибут `lang`. Но вот информации в данном атрибуте не оказалось, было пусто. 
- `keyerror`, `connectionerror`,  `typeerror`, `connecttimeout`, `sslerror`,  `readtimeout`,  `toomanyredirects` -  какая-то причина которая помешала или связаться с сайтом, или забрать нужную информацию. Стоит отметить, что в большинстве случаев основная причина - динамическое формирование контента на сайте и отсутствие опыта по парсингу таких сайтов. 

Попробуем восстановить язык сайта исходя из его домена. Есть предположение, что сайты с доменом `fr`  будут содержать контент на французском. А вот домен `jp` намекает, что контент будет на японском. 

In [404]:
# Обработка пропусков (NaN)
domain.loc[domain['lang']=='', 'lang'] = 'no_lang'

In [406]:
# Выводим в новую метрику запись после последней точки в названии домена (отсекаем поддомены)
domain['after_dot'] = domain['domain'].str.extract(r'.([\w]+$)', expand=True)
domain['after_dot'].unique()

array(['com', 'jp', 'fr', 'ie', 'net', 'ca', 'org', 'uk', 'de', 'se',
       'ch', 'be', 'life', 'nrw', 'co', 'online', 'nl', 'ms', 'dk', 'no',
       'ru', 'info', 'site', 'coop', 'biz', 'nz', 'au', 'us', 'tv', 'fm',
       'eu', 'at', 'network', 'guide', 'it', 'gov', 'studio', 'xyz',
       'news', 'za', 'bm', 'pl', 'hr', 'tokyo', 'style', 'pro', 'cc',
       'cafe', 'p1ai', 'bz', 'blog', 'club', 'mobi', 'ro', 'me', 'ir',
       'su', 'ua', 'edu', 'in', 'social', 'my'], dtype=object)

In [407]:
# Список доменов которые не позволяют однозначно сделать вывод о стране которая предпочтет их использовать
domain_no_country = ['com', 'net', 'org', 'life', 'online', 'info', 'site', 'coop', 'biz', 'tv', 'fm',
                    'network', 'guide', 'gov', 'studio', 'xyz', 'news', 'style', 'pro', 'cafe', 'blog',
                    'club', 'mobi', 'edu', 'social']

In [408]:
# Список ошибок которые получились при сборе данных 
errors = ['keyerror', 'connectionerror', 'no_lang', 'typeerror', 'typeerror', 'sslerror', 'connecttimeout',
         'readtimeout', 'toomanyredirects']

In [409]:
print('Всего доменов: {}'.format(len(domain)),
     '\nДоменов с ошибками: {}'.format(len(domain[domain['lang'].isin(errors)])))

Всего доменов: 6609 
Доменов с ошибками: 1244


Ошибок почти 18%. Как-то много, эх. Произведем замену ошибок на домен последнего уровня и оценим результат.


In [410]:
domain.loc[(domain['lang']=='keyerror')&(~domain['after_dot'].isin(domain_no_country)),'lang'] = domain['after_dot']

### Обработка языков
Посмотрим, что получается по языкам которые удалось собрать.

In [412]:
domain[~domain['lang'].isin(errors)]['lang'].unique()

array(['en', 'ja', 'fr', 'de', 'ca', 'sv', 'se', 'ch', 'nrw', 'nl', 'ms',
       'be', 'da', 'no', 'ru', 'nz', 'at', 'it', 'pl', 'au', 'es', 'eu',
       'ro', 'dk', 'co', 'me', 'pt', 'bg', 'id', 'nb', 'ir', 'ar', 'su',
       'in', 'he'], dtype=object)

- есть дубликаты, например `ru` и `ru-ru`. 
- есть сайты где атрибут `lang` имел сразу два языка.   

Исправим эти разночтения и приведем все к единому типу записи. Сделаем допущение, что если на сайта было указано два языка, то первый считается основным. 

In [413]:
# Промежуточная метрика с первым языком
domain['new_lang'] = domain['lang'].str.extract(r'(..)-')

# Меняем двойные языки на первые
domain.loc[domain['new_lang'].notnull(), 'lang'] = domain['new_lang']

# Удаляем промежуточную метрику
domain.drop(['new_lang'], axis=1, inplace=True)

# Чистим мелкие артефакты
domain.loc[domain['lang']==' da', 'lang'] = 'da'
domain.loc[domain['lang']=='de_de', 'lang'] = 'de'
domain.loc[domain['lang']=='en_us', 'lang'] = 'en'

In [414]:
domain[~domain['lang'].isin(errors)]['lang'].unique()

array(['en', 'ja', 'fr', 'de', 'ca', 'sv', 'se', 'ch', 'nrw', 'nl', 'ms',
       'be', 'da', 'no', 'ru', 'nz', 'at', 'it', 'pl', 'au', 'es', 'eu',
       'ro', 'dk', 'co', 'me', 'pt', 'bg', 'id', 'nb', 'ir', 'ar', 'su',
       'in', 'he'], dtype=object)

Стало лучше, но еще есть что поправить. Так как в рамках данного анализа нет цели различать между собой разные типы английский языков, то объединим их все в единую группу `en`. Заодно исправим помехи от заполнения пропусков доменами. 

In [415]:
domain.loc[domain['lang'].isin(['uk', 'us']), 'lang'] = 'en'

Еще поправим кириллические домены и отнесем их к языку `ru`

In [416]:
domain.loc[domain['lang'].isin(['p1ai']), 'lang'] = 'ru'

А `tokyo` заменим на уже привычный `jp`

In [417]:
domain.loc[domain['lang'].isin(['tokyo']), 'lang'] = 'ja'
domain.loc[domain['lang'].isin(['jp']), 'lang'] = 'ja'

## Выводы
Похоже, что теперь все готово для подведения итогов. Посмотрим как распределяются языки по сайтам изучаемой категории.

In [419]:
print('Всего было найдено языков: {}'.format(len(domain[~domain['lang'].isin(errors)]['lang'].unique())))

Всего было найдено языков: 35


Думаю, что можно смело ограничиться первыми десятью языками

In [420]:
domain[~domain['lang'].isin(errors)]['lang'].value_counts()[:10]

en    4129
ja     462
fr     387
de     195
ru      49
ca      44
nl      17
da      16
sv       9
be       8
Name: lang, dtype: int64

- Абсолютное лидерство английского языка не стало каким-то откровением. Данный результат был вполне ожидаем. Важнее было понять какие языки будут идти сразу за ним. И здесь лидерами стали `ja`, `fr`, `de`.  Стоит отметить, что в данных SimilarWeb совершенно не было сайтов из Китая. Скорее всего это объясняется всем известной изолированностью китайского интернета. 
- Необходимо освоить парсинг сайтов с динамической генерацией контента, пригодится.

In [422]:
# Сохраним данные для потомков
domain.to_csv('../datasets/domain_lang_complete.csv', index=False)